<a href="https://colab.research.google.com/github/giyushino/clip-vit-large-patch14-batch/blob/main/batch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

datasets = load_dataset("cifar10")

labels = datasets["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.16k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/120M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [3]:
import time
import torch
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
def homemade_batch(num_img, batch_size=10, start_img=0):
    homemade = []
    num_batches = num_img // batch_size
    extra = num_img % batch_size

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    t_0 = time.perf_counter()

    for i in range(num_batches):
        t1 = time.perf_counter()

        subset = datasets["train"].select(range((i * batch_size + start_img), (i + 1) * batch_size + start_img))
        input = processor(text=labels, images=subset["img"], return_tensors="pt", padding=False).to(device)
        output = model(**input)

        logits = output.logits_per_image
        probs = logits.softmax(dim=1)

        max_prob, max_index = probs.max(dim=1)
        homemade.append([max_prob.cpu().detach(), max_index.cpu().detach()])
        torch.cuda.empty_cache()

        t2 = time.perf_counter()
        print(f"Finished batch {i + 1} of {num_batches} in {t2 - t1} seconds")

    t_3 = time.perf_counter()
    print(f"Finished entire dataset in {t_3 - t_0} seconds")

    return homemade

In [5]:
def prediction_reformat(subset):
    predicted = []
    count = 1
    for i in range(len(result)):
        for k in range(len(subset[0][0])):
            prob = subset[i][0][k].item()
            id = subset[i][1][k].item()
            label = id2label[id]
            predicted.append([i, label, prob, id])

            count += 1

    return predicted

In [6]:
def accuracy(result):
    correct = 0
    total = 0

    all_labels = {}
    for label in datasets["train"].features["label"].names:
        all_labels[label] = 0

    incorrect = {}
    for label in datasets["train"].features["label"].names:
        incorrect[label] = 0


    for i in range(len(result)):
        all_labels[result[i][1]] += 1
        if datasets["train"][i]["label"] == result[i][3]:
            correct += 1
            total += 1
            if total % 50 == 0:
              print(f"Model accurately predicted {result[i][1]} with {result[i][2] * 100}% confidence.")
        else:
            total += 1
            print(f"Model inaccurately predicted {result[i][1]} with {result[i][2] * 100}% confidence.")
            incorrect[result[i][1]] += 1


    print(f"Acurracy: {(correct/total) * 100}%")

    worst_accuracy = []
    for label in all_labels:
        correct =  all_labels[label] - incorrect[label]
        total = all_labels[label]
        print(f"For {label}: Predicted {correct} out of {total} correct. {(correct) / total * 100}% Accuracy")
        worst_accuracy.append([label, correct/total])

    worst_group = min(worst_accuracy, key=lambda x: x[1])
    print(f"The worst performing group is '{worst_group[0]}' with an accuracy of {worst_group[1] * 100}%")

In [7]:
def data_analysis(predictions):
  cleaned = prediction_reformat(predictions)
  final_results = accuracy(cleaned)

In [8]:
result = homemade_batch(5000)

Finished batch 1 of 500 in 2.282980027999997 seconds
Finished batch 2 of 500 in 0.6410835329999998 seconds
Finished batch 3 of 500 in 0.6535032840000099 seconds
Finished batch 4 of 500 in 0.6460250250000001 seconds
Finished batch 5 of 500 in 0.6440785220000009 seconds
Finished batch 6 of 500 in 0.6472703630000183 seconds
Finished batch 7 of 500 in 0.6265971629999854 seconds
Finished batch 8 of 500 in 0.6259263179999834 seconds
Finished batch 9 of 500 in 0.627985209000002 seconds
Finished batch 10 of 500 in 0.6261290799999983 seconds
Finished batch 11 of 500 in 0.6297125160000121 seconds
Finished batch 12 of 500 in 0.6272678340000084 seconds
Finished batch 13 of 500 in 0.6277880710000261 seconds
Finished batch 14 of 500 in 0.6243518679999909 seconds
Finished batch 15 of 500 in 0.6278038410000022 seconds
Finished batch 16 of 500 in 0.6339276420000317 seconds
Finished batch 17 of 500 in 0.6269675419999885 seconds
Finished batch 18 of 500 in 0.6314157989999671 seconds
Finished batch 19 of 

In [9]:
data_analysis(result)

Model inaccurately predicted automobile with 30.468297004699707% confidence.
Model inaccurately predicted horse with 26.002177596092224% confidence.
Model inaccurately predicted truck with 47.97724783420563% confidence.
Model inaccurately predicted cat with 55.37012219429016% confidence.
Model inaccurately predicted bird with 46.18670046329498% confidence.
Model inaccurately predicted truck with 36.99069321155548% confidence.
Model inaccurately predicted deer with 69.54557299613953% confidence.
Model inaccurately predicted bird with 68.53422522544861% confidence.
Model accurately predicted dog with 80.16961216926575% confidence.
Model inaccurately predicted deer with 63.88816237449646% confidence.
Model inaccurately predicted dog with 86.51031255722046% confidence.
Model inaccurately predicted ship with 31.448206305503845% confidence.
Model inaccurately predicted truck with 69.19840574264526% confidence.
Model inaccurately predicted cat with 88.7103259563446% confidence.
Model accurate